In [3]:
!git clone https://github.com/nerel-ds/NEREL

UsageError: Line magic function `%git` not found.


UsageError: Line magic function `%git` not found.

In [4]:
!unzip NEREL-v1.0.zip -d NEREL

Archive:  NEREL-v1.0.zip
   creating: NEREL-v1.0/
   creating: NEREL-v1.0/dev/
  inflating: NEREL-v1.0/dev/119336_text.ann  
  inflating: NEREL-v1.0/dev/119336_text.txt  
  inflating: NEREL-v1.0/dev/120043_text.ann  
  inflating: NEREL-v1.0/dev/120043_text.txt  
  inflating: NEREL-v1.0/dev/120183_text.ann  
  inflating: NEREL-v1.0/dev/120183_text.txt  
  inflating: NEREL-v1.0/dev/120890_text.ann  
  inflating: NEREL-v1.0/dev/120890_text.txt  
  inflating: NEREL-v1.0/dev/121809_text.ann  
  inflating: NEREL-v1.0/dev/121809_text.txt  
  inflating: NEREL-v1.0/dev/1277254_text.ann  
  inflating: NEREL-v1.0/dev/1277254_text.txt  
  inflating: NEREL-v1.0/dev/1530089_text.ann  
  inflating: NEREL-v1.0/dev/1530089_text.txt  
  inflating: NEREL-v1.0/dev/1535267_text.ann  
  inflating: NEREL-v1.0/dev/1535267_text.txt  
  inflating: NEREL-v1.0/dev/1551405_text.ann  
  inflating: NEREL-v1.0/dev/1551405_text.txt  
  inflating: NEREL-v1.0/dev/1563347_text.ann  
  inflating: NEREL-v1.0/dev/1563347_te

In [5]:
## Read the data# Reading files
from collections import namedtuple
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import nltk

Ann = namedtuple('annotation', ['tag', 'start1', 'end1', 'start2', 'end2', 'text'])
Rel = namedtuple('relationship', ['tag', 'arg1', 'arg2'])

def read_files(folder):
    filenames = sorted(set(e[:e.rfind('.')] for e in os.listdir(folder)))[1:]
    
    texts, entities, relationships  = [], [], []
    for file in tqdm(filenames):
       # if (not file): continue
        path1 = os.path.join(folder, file+'.txt')
        path2 = os.path.join(folder, file+'.ann')
        if (not os.path.exists(path2)):
            print(f'{path2} not found')
            continue
            #with open(path2, 'w'):
            #    pass
            
        with open(path1, 'r', encoding="utf8") as text, open(path2, 'r', encoding="utf8") as ann:
            texts.append(text.read())

            file_entities = {}
            file_relationship = []
            regex_ent = r'T(?P<id>\d+)\s(?P<tag>\w+)\s(?P<start1>\d+) (?P<end1>\d+)(;(?P<start2>\d+) (?P<end2>\d+))?\s(?P<text>.*)'
            regex_rel = r'R(?P<id>\d+)\s(?P<tag>\w+)\sArg1:T(?P<arg1>\d+) Arg2:T(?P<arg2>\d+)'
            
            
            for row in sorted(ann.readlines(), reverse=True):
                #print(row)
                match_ent = re.match(regex_ent, row)
                match_rel = re.match(regex_rel, row)
                if (match_ent):
                    res = match_ent.groupdict()
                    res['start1'] = int(res['start1'])
                    res['end1'] = int(res['end1'])
                    if (res['start2'] is not None):
                        res['start2'] = int(res['start2'])
                        res['end2'] = int(res['end2'])
                    id = res.pop('id')
                    file_entities[id] = Ann(**res)
                elif (match_rel):
                    try:
                        res = match_rel.groupdict()
                        res['arg1'] = file_entities[res['arg1']]
                        res['arg2'] = file_entities[res['arg2']]
                        id = res.pop('id')
                        file_relationship.append(Rel(**res))
                    except KeyError as e:
                        print(f'not found T{e} row={row}')
                else:
                    print(f'incorrect format in: row={row} file={file}')
            entities.append(file_entities)
            relationships.append(file_relationship)
    entities = [sorted(e.values(), key = lambda x: (x.start1, x.end1)) for e in entities]
    return texts, entities, relationships, filenames

In [8]:
folder = 'NEREL/NEREL-v1.0/train'
texts, entities, relationships, filenames = read_files(folder)

incorrect format in: row=По словам очевидцев пешехо
 file=21013_text


100%|██████████| 745/745 [00:01<00:00, 430.97it/s]


In [9]:
def in_range(range1, range2):
    # range1 in range2
    if range1[0] is None: return True
    if range2[0] is None: return False
    return int(range2[0]) <= int(range1[0]) and int(range2[1]) >= int(range1[1])

def is_nested_anns(ent1: Ann, ent2: Ann):
    # ent 1 in ent 2
    res = True
    ent1_range1 = (ent1.start1, ent1.end1)
    ent1_range2 = (ent1.start2, ent1.end2)
    ent2_range1 = (ent2.start1, ent2.end1)
    ent2_range2 = (ent2.start2, ent2.end2)
    #print(f'{ent1_range1=} {ent1_range2=} {ent1_range2=} {ent2_range2=}')
    res = res and (in_range(ent1_range1, ent2_range1) or in_range(ent1_range1, ent2_range2))
    res = res and (in_range(ent1_range2, ent2_range1) or in_range(ent1_range2, ent2_range2))
    return res


def is_nested_anns2(ent1: Ann, ent2: Ann):
    return is_nested_anns(ent1,ent2) or is_nested_anns(ent2, ent1)

def is_nested(rel: Rel):
    return is_nested_anns(rel.arg1, rel.arg2) or is_nested_anns(rel.arg2, rel.arg1)

![](https://i.imgur.com/tgDfc8i.png)             | ![](https://i.imgur.com/oWa5vWo.png)
:-------------------------:|:-------------------------:


In [10]:
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 127 kB 2.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
moto 1.3.14 requires idna<2.9,>=2.5, but you have idna 2.10 which is incompatible.
kaggle 1.5.8 requires urllib3<1.25,>=1.21.1, but you have urllib3 1.25.11 which is incompatible.
aws-sam-translator 1.38.0 requires jsonschema~=3.2, but you have jsonschema 4.1.2 which is incompatible.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [22]:
# #!g1.1
# pre_dataset = []

# for text_id in range(len(texts)):
#     relationships_nested = {(e.arg1, e.arg2):e for e in relationships[text_id] if is_nested(e)}
#     nes = []
#     for i in range(len(entities[text_id])):
#         # O(n^2) eeeeeeeeee
#         for j in range(i+1, len(entities[text_id])):
#             if (is_nested_anns2(entities[text_id][i], entities[text_id][j])):
#                 nes.append((entities[text_id][i], entities[text_id][j]))
#     for e in nes:
#         if (e in relationships_nested):
#             pre_dataset.append((*e, relationships_nested[e].tag))
#         elif ((e[1], e[0]) in relationships_nested):
#             pre_dataset.append((e[1], e[0], relationships_nested[(e[1], e[0])].tag))
#         else:
#             pre_dataset.append((*e, 'None'))
# #             pre_dataset.append((e[1], e[0], 'None'))

# Dataset

In [12]:
#!g1.1
import os
import torch
import time
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from matplotlib import pyplot as plt
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [13]:
#!g1.1
from transformers import BertForSequenceClassification, BertTokenizer

In [14]:
#!g1.1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [15]:
#!g1.1
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

In [17]:
#!g1.1
MAX_LENGTH = 100

In [36]:
#!g1.1
class Binary_Tag:
    def __getitem__(self, x):
        if (x == 'None'): return 0
        else: return 1

# tags = list(set(e.tag for e in sum(relationships,[]))) + ['None'] # ids --> string tag
reversed_tags = Binary_Tag() # string tag --> ids

def tok(text1):
    # longest entity is 36 tokens long
    res = tokenizer.encode_plus(text1,
                      max_length = MAX_LENGTH,
                      pad_to_max_length = True,
                      return_tensors = 'pt',
                      )
    return res['input_ids'][0], res['attention_mask'][0], res['token_type_ids'][0]
    
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, folder):
        texts, entities, relationships, filenames = read_files(folder)
        
        pre_dataset = []

        for text_id in range(len(texts)):
            relationships_nested = {(e.arg1, e.arg2):e for e in relationships[text_id] if is_nested(e)}
            nes = []
            for i in range(len(entities[text_id])):
                # O(n^2) eeeeeeeeee
                for j in range(i+1, len(entities[text_id])):
                    if (is_nested_anns2(entities[text_id][i], entities[text_id][j])):
                        nes.append((entities[text_id][i], entities[text_id][j]))
            for e in nes:
                if (e in relationships_nested):
                    pre_dataset.append((*e, relationships_nested[e].tag))
                elif ((e[1], e[0]) in relationships_nested):
                    pre_dataset.append((e[1], e[0], relationships_nested[(e[1], e[0])].tag))
                else:
                    pre_dataset.append((*e, 'None'))

#         random.seed(2021)
        data = []
        for e1, e2, tag in tqdm(pre_dataset):
            e1, e2 = e1.text, e2.text
            if (len(e1) > len(e2)):
                text = e1.replace(e2, '[ ' + e2 + ' ]')
            else:
                text = e2.replace(e1, '[ ' + e1 + ' ]')
            data.append((reversed_tags[tag], *tok(text)))
        
        self.data = data
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)
    

In [37]:
#!g1.1
train_data = MyDataset('NEREL/NEREL-v1.0/train')
train_dl = DataLoader(train_data, shuffle=True, batch_size=32)

dev_data = MyDataset('NEREL/NEREL-v1.0/dev')
dev_dl = DataLoader(dev_data, batch_size=32)

incorrect format in: row=По словам очевидцев пешехо
 file=21013_text






In [39]:
#!g1.1
next(iter(dev_dl))

[tensor([0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
         1, 1, 0, 1, 1, 1, 1, 0]),
 tensor([[   101,    222,  17710,  ...,      0,      0,      0],
         [   101,  17710,    876,  ...,      0,      0,      0],
         [   101,    222,  13796,  ...,      0,      0,      0],
         ...,
         [   101, 113644,  12333,  ...,      0,      0,      0],
         [   101, 113644,  12333,  ...,      0,      0,      0],
         [   101,    222,  11615,  ...,      0,      0,      0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]])]

In [41]:
#!g1.1
next(iter(dev_dl))[1].size()

torch.Size([32, 100])

# Training

In [44]:
#!g1.1
model = BertForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased-sentence", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, #len(tags), # The number of output labels--18 for our NER dataset
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    max_length = MAX_LENGTH
)

In [46]:
#!g1.1
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [58]:
#!g1.1
def calculate_accuracy(y_pred, y):
    return sum(torch.argmax(y_pred, axis = 1) == y)*1.0/len(y)

def train():
    model.train()
    running_loss = 0
    epoch_accuracy = 0

    pbar = tqdm(enumerate(train_dl), total = len(train_dl))
    for i, (labels, input_ids, attention_mask, token_type_ids) in pbar:
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(input_ids, 
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask, 
                    labels=labels)
        
        loss = outputs[0]
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        epoch_accuracy += calculate_accuracy(outputs[1], labels).item()
        running_loss += loss.item()
        pbar.set_description(f'training: running_loss = {running_loss/(i+1.0):.4f} accuracy = {epoch_accuracy/(i+1.0):.4f},')
    
    scheduler.step(running_loss/(i+1))
     print(f'train loss= {running_loss/(i+1):.4f} \n train accuracy = {epoch_accuracy/(i+1):.4f},')

def test():
    with torch.no_grad():
        model.eval()
        running_loss = 0
        epoch_accuracy = 0

        dev_dl = DataLoader(torch.utils.data.Subset(dev_data, random.sample(range(len(dev_data)), 1000)), batch_size=32)

        pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

        for i, (labels, input_ids, attention_mask, token_type_ids) in pbar:
            labels = labels.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            
            outputs = model(input_ids, 
                        token_type_ids=token_type_ids,
                        attention_mask=attention_mask, 
                        labels=labels)
            

            loss = outputs[0]
            calculate_accuracy(outputs[1], labels)

            running_loss += loss.item()
            epoch_accuracy += calculate_accuracy(outputs[1], labels).item()
            pbar.set_description(f'testing: running_loss = {running_loss/(i+1):.4f} accuracy = {epoch_accuracy/(i+1):.4f},')
        print(f'test loss= {running_loss/(i+1):.4f} \n test accuracy = {epoch_accuracy/(i+1):.4f},')

In [51]:
#!g1.1
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate 
                  eps = 5e-8#1e-8 # args.adam_epsilon 
                )
scheduler = ReduceLROnPlateau(optimizer, patience=5, cooldown = 1, factor = 0.5)

In [56]:
#!g1.1
seed_val = 2021

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
for epoch in range(1, 6):
    print('lr=', optimizer.param_groups[0]['lr'])
    train()
    test()



test loss= 0.3357 
 test accuracy = 0.8701,


test loss= 0.3253 
 test accuracy = 0.8672,


test loss= 0.3959 
 test accuracy = 0.8604,


test loss= 0.3997 
 test accuracy = 0.8496,


test loss= 0.4966 
 test accuracy = 0.8555,


In [60]:
#!g1.1
for epoch in range(6, 11):
    print(f'epoch = {epoch}')
    print('lr=', optimizer.param_groups[0]['lr'])
    train()
    test()

epoch = 6


test loss= 0.5340 
 test accuracy = 0.8525,
epoch = 7


test loss= 0.6721 
 test accuracy = 0.8369,
epoch = 8


test loss= 0.6574 
 test accuracy = 0.8467,
epoch = 9


test loss= 0.4790 
 test accuracy = 0.8721,
epoch = 10


test loss= 0.5836 
 test accuracy = 0.8535,


In [61]:
#!g1.1
torch.save(model.state_dict(), 'models/Bert_binary_nested_only')

In [72]:
#!g1.1
from sklearn.metrics import f1_score

with torch.no_grad():
    model.eval()
    running_loss = 0
    epoch_accuracy = 0

    dev_dl = DataLoader(dev_data, batch_size=32)

    pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

    y_true = []
    y_pred = []

    for i, (labels, input_ids, attention_mask, token_type_ids) in pbar:
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)

        outputs = model(input_ids, 
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask, 
                    labels=labels)


        loss = outputs[0]
        calculate_accuracy(outputs[1], labels)
        y_true += [e.item() for e in labels]
        y_pred += [e.item() for e in torch.argmax(outputs[1], axis = 1)]

        running_loss += loss.item()
        epoch_accuracy += calculate_accuracy(outputs[1], labels).item()
        # pbar.set_description(f'testing: running_loss = {running_loss/(i+1):.4f} accuracy = {epoch_accuracy/(i+1):.4f},')
    print(f'test loss= {running_loss/(i+1):.4f} \n test accuracy = {epoch_accuracy/(i+1):.4f}, F1 = {f1_score(y_true, y_pred)}')


test loss= 0.5690 
 test accuracy = 0.8593, F1 = 0.8268684957426679


/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:848: UserWarning: The following variables cannot be serialized: pbar
  warnings.warn(message)


In [73]:
#!g1.1
for epoch in range(11, 21):
    print(f'epoch = {epoch}')
    print('lr=', optimizer.param_groups[0]['lr'])
    train()
    test()

epoch = 11


test loss= 0.6326 
 test accuracy = 0.8408,
epoch = 12


test loss= 0.5228 
 test accuracy = 0.8564,
epoch = 13


test loss= 0.5662 
 test accuracy = 0.8682,
epoch = 14


test loss= 0.4771 
 test accuracy = 0.8564,
epoch = 15


test loss= 0.5958 
 test accuracy = 0.8682,
epoch = 16


test loss= 0.5555 
 test accuracy = 0.8652,
epoch = 17


test loss= 0.6109 
 test accuracy = 0.8613,
epoch = 18


test loss= 0.4904 
 test accuracy = 0.8457,
epoch = 19


test loss= 0.6501 
 test accuracy = 0.8496,
epoch = 20


test loss= 0.6386 
 test accuracy = 0.8545,


In [74]:
#!g1.1
print('lr=', optimizer.param_groups[0]['lr'])

lr= 5e-05


In [75]:
#!g1.1
from sklearn.metrics import f1_score

with torch.no_grad():
    model.eval()
    running_loss = 0
    epoch_accuracy = 0

    dev_dl = DataLoader(dev_data, batch_size=32)

    pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

    y_true = []
    y_pred = []

    for i, (labels, input_ids, attention_mask, token_type_ids) in pbar:
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)

        outputs = model(input_ids, 
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask, 
                    labels=labels)


        loss = outputs[0]
        calculate_accuracy(outputs[1], labels)
        y_true += [e.item() for e in labels]
        y_pred += [e.item() for e in torch.argmax(outputs[1], axis = 1)]

        running_loss += loss.item()
        epoch_accuracy += calculate_accuracy(outputs[1], labels).item()
        # pbar.set_description(f'testing: running_loss = {running_loss/(i+1):.4f} accuracy = {epoch_accuracy/(i+1):.4f},')
    print(f'test loss= {running_loss/(i+1):.4f} \n test accuracy = {epoch_accuracy/(i+1):.4f}, F1 = {f1_score(y_true, y_pred)}')


test loss= 0.6290 
 test accuracy = 0.8601, F1 = 0.8194444444444444


/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:848: UserWarning: The following variables cannot be serialized: pbar
  warnings.warn(message)


In [76]:
#!g1.1
torch.save(model.state_dict(), 'models/Bert_binary_nested_only-2')

In [99]:
#!g1.1
from sklearn.metrics import f1_score

with torch.no_grad():
    model.eval()
    running_loss = 0
    epoch_accuracy = 0

    dev_dl = DataLoader(dev_data, batch_size=1)

    pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

    y_true = []
    y_pred = []

    for i, (labels, input_ids, attention_mask, token_type_ids) in pbar:
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)

        outputs = model(input_ids, 
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask, 
                    labels=labels)


        loss = outputs[0]
#         print(input_ids[0].detach().cpu().numpy())
        if (torch.argmax(outputs[1], axis = 1) != labels):
            print('expected:', labels.detach().cpu().item())
            print('found:', torch.argmax(outputs[1], axis = 1).detach().cpu().item(), '(', outputs[1].detach().cpu().numpy() ,')')
            print(tokenizer.decode(input_ids[0].detach().cpu().numpy()).replace(r'[PAD]', ''))

expected: 0
found: 1 ( [[-2.3732822  1.5984825]] )
[CLS] [ Строгановское ] высшее художественное училище [SEP]                                                                                           
expected: 0
found: 1 ( [[-1.2307636  0.9554411]] )
[CLS] главным менеджером Международного музыкального фестиваля « [ Ереванские ] перспективы » [SEP]                                                                                      
expected: 0
found: 1 ( [[-2.1465607  1.4853911]] )
[CLS] главным менеджером Международного музыкального фестиваля « [ Ереванские перспективы ] » [SEP]                                                                                      
expected: 0
found: 1 ( [[-2.3502862  1.587475 ]] )
[CLS] Международного музыкального фестиваля « [ Ереванские ] перспективы [SEP]                                                                                         
expected: 0
found: 1 ( [[-2.383545   1.6036669]] )
[CLS] Премьер - министр [ Великобритании ] [SEP]      

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:848: UserWarning: The following variables cannot be serialized: pbar
  warnings.warn(message)
